![Image](resources/banner.jpg)

<h1 align="center">Cell Type Lookup Table </h1> 
<h2 align="center"> Day 1, Morning Session. SWDB 2024 </h2> 

<h3 align="center">Monday, August 19, 2023</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
This notebook provides examples of how to load, examine, and visualize electrophysiological data containing several known cell types. This data comes from the Cell Type Lookup Table dataset, which contains **extracallular electrophysiological** data with specific cell types identified via **optotagging**. This data set in particular was recorded from the striatum, looking at the responses of D1, D2, and cholinergic neurons. Every session has two of these three cell types tagged.

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
***What kind of questions can you answer with this dataset?***

This dataset contains recordings of activity from known and identified cell types. In particular, every session contains two different, simultaneously tagged cell types. This makes it suitable for a variety of circuit architecture and connectivity questions.

- How are stimuli and features from the external world encoded in neural responses? How do the encoding properties differ across cell types?
- Are different cell types differentially predictive of behavior?
- Do different cell types show different levels of connectivity between each other and other cell types?
- How do neurons within and across cell types coordinate their activity? 
- Is neural information differentially stored by cell type?

These are just some of the questions that might be addressed from this type of data.  

</div>

In [ ]:
from hdmf_zarr import NWBZarrIO
import os, platform

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Unlike other Allen Institute datasets, we interact with these NWBs files directly rather than through the SDK. We will be using `hdmf_zarr` to interact with this dataset. See https://allenswdb.github.io/physiology/pyNWB/pyNWB.html for more details.

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Load an NWB

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Let's try loading and exploring one session's worth of data.

</div>

In [ ]:
# Set file location based on platform. 
platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2024/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on Code Ocean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/"

In [ ]:
# an 'arbitrarily' selected session
session = '661398_2023-04-03_15-47-29'
session_directory = os.path.join(data_root,f'SWDB 2024 CTLUT data/ecephys_{session}_nwb')

nwb_file = os.path.join(session_directory, f'ecephys_{session}_experiment1_recording1.nwb.zarr')
io = NWBZarrIO(nwb_file, "r")
nwbfile_read = io.read()

You can use the **nwbwidgets** module to visually explore the contents of an nwb file.

In [ ]:
from nwbwidgets import nwb2widget
nwb2widget(nwbfile_read)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Load the units table

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
The units table contains all the "neurons" that were detected by the spike sorter (in this case, Kilosort 2.5). We usually refer to them as "units" because we can't actually be completely certain that the spike sorter output corresponds 1-to-1 to the actual neurons that were present at the time of the recording: this is just our best guess!

</div>

In [ ]:
units = nwbfile_read.units[:]
units.head()

In [ ]:
units.columns

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
You may notice this units table has a lot of columns! Many of these are quality metrics or laser response metrics, which are more fully explained in the databook. You can use these to filter out units that are unlikely to be real neurons or units whose data is too noisy for analysis. Among the most interesting columns is the **spike_times** column, which tells us every time point when this "neuron" fired an "action potential."

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Let's also take a look at the experimental epochs that were present in this session.

</div>

In [ ]:
nwbfile_read.epochs[:]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
We see three epochs: a pre-stimulus time period where spontaneous activity was recorded, a stimulus period where laser presentations took place in order to identify taed neurons, and a very short post-stimulus period that is basically just the amount of time it took to shut down the experiment.

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Let's try plotting all the units in a raster plot, to see what activity looked like durin the experiment!

</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

exp_start_time = nwbfile_read.epochs.start_time[0]
exp_stop_time = nwbfile_read.epochs.stop_time[2]
start_time_laser = nwbfile_read.epochs.start_time[1]
stop_time_laser = nwbfile_read.epochs.stop_time[1]

for unit_id, unit_row in units.iterrows():
    unit_spiketimes = unit_row.spike_times
    plt.plot(unit_spiketimes, np.tile(unit_id, len(unit_spiketimes)), 'k.', ms=1, alpha=0.03)

# add a shaded bit to indicate the laser presentation epoch
ax = plt.gca()
yLims = [0, unit_id]
plt.ylim(yLims)
plt.xlim([exp_start_time, exp_stop_time])
rect = patches.Rectangle((start_time_laser, yLims[0]), stop_time_laser - start_time_laser, yLims[1] - yLims[0], linewidth=1, edgecolor='tomato', facecolor='tomato', alpha=0.35, clip_on=False)
ax.add_patch(rect)

plt.xlabel('Time (s)')
plt.ylabel('Unit ID')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
You can see that unit activity increases a lot during the laser presentation epoch, which makes sense: you're shining lasers into a mouse's brain! To get a better read on what the units are doing in more natural circumstances, let's take a look at the pre-laser-stimulation activity:

</div>

In [ ]:
start_time = nwbfile_read.epochs.start_time[0]
stop_time = nwbfile_read.epochs.stop_time[0]

for unit_id, unit_row in units.iterrows():
    unit_spiketimes = unit_row.spike_times
    pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]
    plt.plot(pre_stim_spiketimes, np.tile(unit_id, len(pre_stim_spiketimes)), 'k.', ms=1, alpha=0.1)
    
plt.xlabel('Time (s)')
plt.ylabel('Unit ID')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
However, a lot of these "neurons" are probably noise or artifacts. We can use some of the quality metrics contained in the units table to filter out some of the "bad" "neurons." A common way to curate units is by the amount of ISI (interspike interval) violations. Neurons physiologically can not spike more often than every 2 ms, so a high ratio of "spikes" that happen less than 2 ms apart is a good indicator that this unit is not a real neuron (or, at least, not a *single* neuron).

Another way units are curated is by visually inspecting the spike waveforms to determine if they look like action potentials or just electrical noise. This table contains the results on someone's manual curation of the waveforms in the column *noise_label*.

</div>

In [ ]:
# query the units table to get units with good waveforms and low ISI violations!
good_units = units.query("isi_violations_ratio < 0.5 and noise_label == 'good'")

# and now plot only the "good" units
for unit_id, unit_row in good_units.iterrows():
    unit_spiketimes = unit_row.spike_times
    pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]
    plt.plot(pre_stim_spiketimes, np.tile(unit_id, len(pre_stim_spiketimes)), 'k.', ms=1, alpha=0.1)
    
plt.xlabel('Time (s)')
plt.ylabel('Unit ID')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Task 1**: The most common standard QC criteria used at the Allen Institute are isi_violations_ratio < 0.5, amplitude_cutoff < 0.1, and presense_ratio > 0.8. Try querying for all three of these criteria and plotting the resulting *really* good units!

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Many of these nwb files also contain units from multiple probes concatenated together. Let's see how many probes were used in this recording.

</div>

In [ ]:
probes = np.unique(units.device_name)
probes

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
It might be more useful to have separate plots for the two probes.

</div>

In [ ]:
for ind_probe, probe in enumerate(probes):
    plt.subplot(1,2,ind_probe+1)
    this_probe_units = good_units.query("device_name == @probe")
    
    for unit_id, unit_row in this_probe_units.iterrows():
        unit_spiketimes = unit_row.spike_times
        pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]
        plt.plot(pre_stim_spiketimes, np.tile(unit_id, len(pre_stim_spiketimes)), 'k.', ms=1, alpha=0.1)
        plt.title(probe)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Task 2**: The *peak_channel* column contains the electrode closest to the unit, as determined by the peak waveform. This gives us a location for the unit. Try plotting the above rasters with *peak_channel* as the y-axis rather than *unit_id*, which is assigned somewhat arbitrarily!

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Identify tagged units

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
During this experiment, known cell types were identified by their responses to pulses of laser. You can learn more about this technique in the databook (https://allenswdb.github.io/background/Optotagging.html)! The units table contains laser response metrics for every units, as well as a `predicted_cell_type` label. Let's try filtering the units table by `predicted_cell_type`!

</div>

In [ ]:
np.unique(units.predicted_cell_type)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
This experiment was tagging D1 and D2 medium spiny neurons in the striatum! We can filter the units table for units for which we have a predicted cell type. Note that just because a unit is labeled as "untagged," that does not guarantee it *isn't* a D1 or D2 cell. We just don't know what it is.

</div>

In [ ]:
tagged_units = units.query("predicted_cell_type != 'untagged'")
tagged_units.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Let's see what the activity of just these units looks like!

</div>

In [ ]:
cell_type_colors = {'D1' : 'skyblue', 'D2' : 'tomato'}

for ind, (unit_id, unit_row) in enumerate(tagged_units.iterrows()):
    unit_spiketimes = unit_row.spike_times
    pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]

    # let's try making a psth of the response instead of a raster!
    bin_length = 5
    time_bins = np.arange(np.floor(start_time),np.ceil(stop_time),bin_length)
    h,v = np.histogram(unit_spiketimes, time_bins)
    plt.plot(v[:-1],h/bin_length, color=cell_type_colors[unit_row.predicted_cell_type])
    
plt.xlabel('Time (s)')
plt.ylabel('Firing rate (spikes/s)')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Task 3**: cell type predictions are based only on the unit's laser reponses, not the quality of the unit. There's a good chance many of these are laser artifacts or other types of noise or multiunit activity. Try further filtering the units table for tagged, *quality* units.

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Load running speed

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
Some behavioral data was collected during this experiment in addition to the physiology. For instance, the mouse was on a running wheel with an encoder, and so we recorded the mouse's running speed for the entire experiment. Let's try loading that data!

</div>

In [ ]:
running = nwbfile_read.processing['behavior']['BehavioralTimeSeries']['linear velocity']
running_speed = np.array(running.data)
running_timestamps = np.array(running.timestamps)

In [ ]:
# plot the running speed for the same time period as the rasters above
pre_stim_timestamps = running_timestamps[running_timestamps < stop_time]
pre_stim_speed = running_speed[running_timestamps < stop_time]

plt.plot(pre_stim_timestamps, pre_stim_speed)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Task 4**: Try plotting the running speed and the units raster in the same figure with a shared x-axis. Does any of the neuronal activity seem related to the mouse's movement?

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

***Exercise 1***

Pick a session from the cell type lookup table data set and identify all the quality, tagged units. Do the units of different cell types differ from each other in any simple metric, such as average firing rate during the pre-stimulus period? 

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

***Exercise 2***

Identify quality, tagged units of the same cell type from multiple sessions. Is the activity of this cell type consistent across sessions? 

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

***Exercise 3***

Convince yourself a tagged, quality unit really is tagged by creating a raster plot of its response to laser stimulation. You will need the trials table for this to get the onsets and identities of the laser trials!  

</div>